In [ ]:
Selección de campos

In [ ]:
# Importación de bibliotecas necesarias
import pandas as pd  # Para manipulación y análisis de datos estructurados
import numpy as np  # Para operaciones numéricas
import random  # Para generar números aleatorios
from sklearn.model_selection import train_test_split  # Para dividir datos en conjuntos de entrenamiento y prueba
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # Para preprocesamiento de datos
from sklearn.ensemble import RandomForestRegressor  # Algoritmo de regresión basado en árboles de decisión
from sklearn.metrics import mean_squared_error, r2_score  # Métricas para evaluar el rendimiento del modelo
from transformers import BertTokenizer, BertModel  # Modelo BERT para procesamiento de lenguaje natural
import torch  # Framework para redes neuronales y computación tensorial

# Cargar modelo y tokenizer de BERT
# BERT es un modelo preentrenado de lenguaje natural desarrollado por Google
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Convierte texto en tokens que BERT puede procesar
bert_model = BertModel.from_pretrained('bert-base-uncased')  # Modelo que genera representaciones contextuales de texto

def get_bert_embeddings(texts):
    """Convierte una lista de textos en embeddings de BERT.
    
    Los embeddings son representaciones numéricas de texto que capturan significado semántico.
    BERT produce embeddings contextuales de 768 dimensiones por token.
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
    with torch.no_grad():  # Desactivamos cálculo de gradientes para ahorrar memoria
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Extraemos solo el embedding del token [CLS]

# Ejemplos de proyectos por categoría
# Este diccionario contiene datos estructurados para ejemplos de proyectos en diferentes sectores
# Cada proyecto tiene un resumen, problema que aborda, solución propuesta y diferenciación
project_details = {
    'Tech': [
        {
            'summary': "Plataforma de pagos transfronterizos que facilita la transferencia de dinero de manera rápida y segura.",
            'problem_addressed': "Los pagos internacionales son costosos y lentos, afectando a las pequeñas empresas.",
            'solution_proposed': "Solución que ofrece tarifas de pago transfronterizas reducidas y procesamiento rápido, utilizando blockchain.",
            'differentiation': "Utiliza una red de blockchain descentralizada para garantizar la transparencia y baja de costos en pagos."
        },
        {
            'summary': "Aplicación de microinversiones que permite a usuarios invertir pequeñas cantidades en bolsa.",
            'problem_addressed': "El acceso a inversiones en bolsa requiere capital inicial alto y conocimientos financieros avanzados.",
            'solution_proposed': "Plataforma que permite inversiones desde 1€ y automatiza decisiones basadas en perfiles de riesgo.",
            'differentiation': "Interfaz intuitiva con recomendaciones personalizadas basadas en machine learning."
        },
        {
            'summary': "Plataforma de préstamos P2P que conecta directamente a prestatarios con inversores.",
            'problem_addressed': "Dificultad para acceder a créditos por parte de pequeños emprendedores y altas tasas de interés bancarias.",
            'solution_proposed': "Sistema que evalúa el riesgo crediticio mediante IA y conecta con inversores dispuestos a financiar.",
            'differentiation': "Evaluación de riesgo basada en datos alternativos más allá del historial crediticio tradicional."
        }
    ],
    'Energía': [
        {
            'summary': "Desarrollo de un sistema de energía solar de bajo costo para hogares en zonas rurales.",
            'problem_addressed': "El acceso a energía limpia en zonas rurales es limitado y costoso.",
            'solution_proposed': "Sistema de paneles solares con batería de almacenamiento que reduce el costo de energía en áreas rurales.",
            'differentiation': "Baterías solares desarrolladas con tecnología avanzada que optimizan la eficiencia en lugares remotos."
        },
        {
            'summary': "Red inteligente para optimización del consumo eléctrico en edificios comerciales.",
            'problem_addressed': "El desperdicio energético en grandes edificios genera altos costos y emisiones innecesarias.",
            'solution_proposed': "Sistema IoT que monitoriza y ajusta automáticamente el consumo energético según uso y ocupación.",
            'differentiation': "Algoritmos predictivos que anticipan necesidades energéticas y reducen consumo hasta un 40%."
        },
        {
            'summary': "Generadores eólicos de pequeña escala para entornos urbanos.",
            'problem_addressed': "Las energías renovables como la eólica no están adaptadas para uso en ciudades y espacios residenciales.",
            'solution_proposed': "Microturbinas eólicas que se adaptan a edificios y generan electricidad con vientos de baja intensidad.",
            'differentiation': "Diseño silencioso y estético que permite su instalación en zonas residenciales sin impacto visual."
        }
    ],
    'Salud': [
        {
            'summary': "Aplicación de telemedicina que conecta a pacientes con médicos especializados en tiempo real.",
            'problem_addressed': "El acceso a atención médica de calidad en zonas remotas es insuficiente.",
            'solution_proposed': "Aplicación que permite consultas médicas en línea, reduciendo los tiempos de espera y costos de consulta.",
            'differentiation': "Ofrece consultas médicas en línea las 24 horas, sin necesidad de desplazamientos."
        },
        {
            'summary': "Dispositivo wearable para monitorización continua de pacientes con enfermedades crónicas.",
            'problem_addressed': "El seguimiento de pacientes crónicos es costoso y requiere visitas frecuentes a centros médicos.",
            'solution_proposed': "Dispositivo no invasivo que monitoriza constantes vitales y envía alertas automáticas a médicos.",
            'differentiation': "Integra sensores avanzados con IA para detectar anomalías antes de que se conviertan en emergencias."
        },
        {
            'summary': "Sistema de diagnóstico por imagen asistido por IA para detección temprana de cáncer.",
            'problem_addressed': "El diagnóstico tardío de diversos tipos de cáncer reduce significativamente las tasas de supervivencia.",
            'solution_proposed': "Software que analiza imágenes médicas y detecta patrones sospechosos con mayor precisión que métodos tradicionales.",
            'differentiation': "Algoritmo entrenado con millones de imágenes que supera en un 30% la precisión de diagnósticos humanos."
        }
    ],
    'Educación': [
        {
            'summary': "Herramienta educativa basada en IA para personalizar los métodos de enseñanza según el perfil del estudiante.",
            'problem_addressed': "El sistema educativo no está personalizado y no responde a las necesidades individuales de los estudiantes.",
            'solution_proposed': "Plataforma educativa basada en IA que adapta los contenidos a las necesidades y ritmo del estudiante.",
            'differentiation': "Personaliza el aprendizaje mediante algoritmos que se ajustan a la forma de aprender del estudiante."
        },
        {
            'summary': "Plataforma de aprendizaje de idiomas mediante realidad virtual inmersiva.",
            'problem_addressed': "Los métodos tradicionales de aprendizaje de idiomas no logran contextos reales de práctica conversacional.",
            'solution_proposed': "Experiencias de realidad virtual que simulan entornos reales para practicar idiomas con nativos virtuales.",
            'differentiation': "Tecnología de reconocimiento de voz que analiza la pronunciación y ofrece correcciones en tiempo real."
        },
        {
            'summary': "Sistema de microcredenciales digitales para validar habilidades profesionales específicas.",
            'problem_addressed': "La brecha entre educación formal y habilidades requeridas en el mercado laboral actual.",
            'solution_proposed': "Plataforma que certifica habilidades específicas mediante pruebas prácticas verificadas por empleadores.",
            'differentiation': "Credenciales basadas en blockchain que son inmutables y verificables instantáneamente por reclutadores."
        }
    ]
}

# Generación de datos sintéticos con ejemplos reales
def generate_synthetic_data(n=500):
    """
    Genera un conjunto de datos sintéticos de proyectos basados en los ejemplos reales.
    
    Args:
        n: Número de muestras a generar (por defecto 500)
        
    Returns:
        DataFrame con datos sintéticos de proyectos y sus características
    """
    # Definición de valores posibles para variables categóricas
    categories = ['Tech', 'Energía', 'Salud', 'Educación']
    stages = ['Idea', 'Prototipo', 'MVP', 'Escalando']
    asset_types = ['Equity', 'Deuda', 'Otro']
    legal_statuses = ['Regulado', 'No regulado']
    impact_types = ['Social', 'Ambiental', 'Económico']
    
    # Listas para almacenar los datos generados
    category_list = []
    summary_list = []
    problem_list = []
    solution_list = []
    differentiation_list = []
    
    # Generar datos basados en ejemplos reales
    for _ in range(n):
        # Seleccionar una categoría aleatoria 
        category = np.random.choice(categories)
        category_list.append(category)
        
        # Obtener la lista de proyectos disponibles para esta categoría
        available_projects = project_details[category]
        # Seleccionar aleatoriamente un proyecto específico de esta categoría
        example = random.choice(available_projects)
        
        # Extraer textos del ejemplo seleccionado
        summary_list.append(example['summary'])
        problem_list.append(example['problem_addressed'])
        solution_list.append(example['solution_proposed'])
        
        # Verificar y extraer diferenciación si existe
        if 'differentiation' in example:
            differentiation_list.append(example['differentiation'])
        else:
            differentiation_list.append("Ventaja competitiva en el mercado")
    
    # Crear DataFrame con todos los datos
    data = pd.DataFrame({
        'category': category_list,
        'stage': np.random.choice(stages, n),  # Etapa del proyecto (aleatoria)
        'summary': summary_list,
        'problem_addressed': problem_list,
        'solution_proposed': solution_list,
        'differentiation': differentiation_list,
        'tokenized_asset_type': np.random.choice(asset_types, n),  # Tipo de activo (aleatorio)
        'asset_existing': np.random.choice([0, 1], n),  # Si el activo existe o no (binario)
        'legal_status': np.random.choice(legal_statuses, n),  # Estado legal (aleatorio)
        'estimated_total_cost_eur': np.random.uniform(50000, 1000000, n),  # Costo estimado (aleatorio entre 50k-1M)
        'funding_requested_eur': np.random.uniform(10000, 500000, n),  # Financiación solicitada (aleatorio entre 10k-500k)
        'expected_annual_revenue_eur': np.random.uniform(20000, 2000000, n),  # Ingresos esperados (aleatorio entre 20k-2M)
        'current_clients_or_pilots': np.random.choice([0, 1], n),  # Si tiene clientes actuales (binario)
        'scalability_score': np.random.randint(1, 6, n),  # Puntuación de escalabilidad (1-5)
        'impact_type': np.random.choice(impact_types, n),  # Tipo de impacto (aleatorio)
    })
    
    # Crear una puntuación de viabilidad más realista basada en las características
    # La fórmula combina varios factores que afectan la viabilidad del proyecto:
    
    # Factor 1: La etapa más avanzada aumenta la viabilidad
    stage_score = data['stage'].map({'Idea': 10, 'Prototipo': 20, 'MVP': 30, 'Escalando': 40})
    
    # Factor 2: La existencia de clientes aumenta la viabilidad
    client_score = data['current_clients_or_pilots'] * 15
    
    # Factor 3: Mayor escalabilidad aumenta la viabilidad
    scalability_score = data['scalability_score'] * 5
    
    # Factor 4: Mejor ratio ingresos/costos aumenta la viabilidad
    revenue_ratio = np.clip(data['expected_annual_revenue_eur'] / (data['estimated_total_cost_eur'] + 1), 0, 5) * 7
    
    # Calcular puntuación final con algo de ruido aleatorio para simular factores desconocidos
    data['viability_score'] = stage_score + client_score + scalability_score + revenue_ratio + np.random.normal(0, 5, n)
    
    # Limitar los valores entre 0 y 100 para mantenerlo en un rango interpretable
    data['viability_score'] = np.clip(data['viability_score'], 0, 100)
    
    return data

# Crear datos sintéticos para el entrenamiento y evaluación
data = generate_synthetic_data()

# Separar variables por tipo para aplicar diferentes preprocesamientos
categorical_cols = ['category', 'stage', 'tokenized_asset_type', 'legal_status', 'impact_type']  # Variables categóricas
text_cols = ['summary', 'problem_addressed', 'solution_proposed', 'differentiation']  # Variables de texto
numeric_cols = ['estimated_total_cost_eur', 'funding_requested_eur', 'expected_annual_revenue_eur', 'scalability_score']  # Variables numéricas
binary_cols = ['asset_existing', 'current_clients_or_pilots']  # Variables binarias (0/1)

# Preprocesamiento de variables categóricas mediante codificación one-hot
# One-hot convierte categorías en vectores binarios (ej: ['A','B','C'] -> [1,0,0], [0,1,0], [0,0,1])
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorical_features = encoder.fit_transform(data[categorical_cols])

# Procesar cada columna de texto individualmente usando BERT
text_features = []
for col in text_cols:
    # Obtener embeddings para cada columna de texto
    col_embeddings = get_bert_embeddings(data[col].tolist())
    text_features.append(col_embeddings)

# Concatenar todos los embeddings de texto horizontalmente
text_features_combined = np.hstack(text_features)

# Normalización de variables numéricas (media 0, desviación estándar 1)
# Esto evita que variables con valores grandes dominen sobre otras
scaler = StandardScaler()
numeric_features = scaler.fit_transform(data[numeric_cols])

# Unir todas las características en una matriz X
X = np.hstack([categorical_features, text_features_combined, numeric_features, data[binary_cols].values])
# La variable objetivo y es la puntuación de viabilidad
y = data['viability_score']

# Separar datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo Random Forest con hiperparámetros optimizados
# Random Forest es un conjunto de árboles de decisión que promedia sus predicciones
model = RandomForestRegressor(
    n_estimators=200,  # Número de árboles en el bosque
    max_depth=15,      # Profundidad máxima de cada árbol
    min_samples_split=5,  # Mínimo de muestras para dividir un nodo
    min_samples_leaf=2,   # Mínimo de muestras en cada nodo hoja
    random_state=42       # Semilla para reproducibilidad
)
model.fit(X_train, y_train)  # Entrenamiento del modelo

# Evaluación del modelo en el conjunto de prueba
y_pred = model.predict(X_test)  # Predicciones en datos de prueba
mse = mean_squared_error(y_test, y_pred)  # Error cuadrático medio (menor es mejor)
r2 = r2_score(y_test, y_pred)  # Coeficiente de determinación (mayor es mejor, máx=1)

print(f'MSE: {mse:.2f}')  # Muestra el error cuadrático medio
print(f'R²: {r2:.2f}')    # Muestra el coeficiente R²

# Mostrar importancia de características para interpretar el modelo
feature_importances = model.feature_importances_
print("\nCaracterísticas más importantes:")
# Mostrar las 10 características más importantes
top_n = 10
indices = np.argsort(feature_importances)[-top_n:]
for i in reversed(indices):
    print(f"Característica {i}: {feature_importances[i]:.4f}")

# Modelo sin BERT para comparación y evaluar el impacto de las características de texto
X_without_text = np.hstack([categorical_features, numeric_features, data[binary_cols].values])
X_train_without_text, X_test_without_text, y_train_without_text, y_test_without_text = train_test_split(X_without_text, y, test_size=0.2, random_state=42)

# Entrenar segundo modelo sin usar texto
model_without_text = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model_without_text.fit(X_train_without_text, y_train_without_text)

# Evaluar el segundo modelo (sin BERT)
y_pred_without_text = model_without_text.predict(X_test_without_text)
mse_without_text = mean_squared_error(y_test_without_text, y_pred_without_text)
r2_without_text = r2_score(y_test_without_text, y_pred_without_text)

print(f'\nMSE without BERT: {mse_without_text:.2f}')
print(f'R² without BERT: {r2_without_text:.2f}')

print('================================================')
print('================================================')
print('================================================')

# Crear nuevos ejemplos para validación adicional del modelo
# Estos ejemplos son diferentes a los de entrenamiento para probar generalización
validation_examples = {
    'Tech': [
        {
            'summary': "Plataforma de gestión financiera personal con inteligencia artificial para optimizar las finanzas del hogar.",
            'problem_addressed': "La mayoría de las personas no tienen visibilidad clara de sus patrones de gasto ni herramientas para mejorar sus hábitos financieros.",
            'solution_proposed': "Software que analiza automáticamente los gastos, detecta patrones, y propone estrategias de ahorro personalizadas.",
            'differentiation': "Algoritmos de IA que predicen futuras dificultades financieras y sugieren acciones preventivas con tres meses de anticipación."
        },
        {
            'summary': "Seguro paramétrico basado en blockchain para pequeños agricultores contra eventos climáticos extremos.",
            'problem_addressed': "Los agricultores en países en desarrollo carecen de acceso a seguros tradicionales contra desastres naturales que afectan sus cosechas.",
            'solution_proposed': "Sistema de seguro automático que paga directamente al agricultor cuando sensores meteorológicos registran condiciones extremas.",
            'differentiation': "Contratos inteligentes que eliminan los trámites de reclamación y reducen costos operativos hasta en un 85%."
        }
    ],
    'Energía': [
        {
            'summary': "Sistema de almacenamiento de energía basado en hidrogeno verde para comunidades aisladas.",
            'problem_addressed': "Las comunidades remotas dependen de generadores diésel contaminantes y costosos para su suministro energético.",
            'solution_proposed': "Instalación que genera hidrógeno mediante electrólisis con energía renovable y lo almacena para uso cuando no hay sol o viento.",
            'differentiation': "Tecnología de electrolizadores de alta eficiencia que funciona con agua de baja calidad y reduce costos en un 40%."
        },
        {
            'summary': "Plataforma de comercio de energía P2P para comunidades energéticas locales.",
            'problem_addressed': "Los prosumidores (productores-consumidores) de energía solar no pueden comercializar eficientemente sus excedentes energéticos.",
            'solution_proposed': "Marketplace digital que permite a vecinos comprar y vender excedentes de energía renovable dentro de su comunidad local.",
            'differentiation': "Sistema de precios dinámicos basado en IA que maximiza el ahorro para compradores y el beneficio para vendedores."
        }
    ],
    'Salud': [
        {
            'summary': "Plataforma de diagnóstico remoto para zonas rurales basada en dispositivos médicos portátiles.",
            'problem_addressed': "Más de 2 mil millones de personas en el mundo carecen de acceso a servicios de diagnóstico médico básico.",
            'solution_proposed': "Kit que incluye dispositivos de análisis de sangre, ecografía y ECG portátiles conectados a smartphones para telemedicina.",
            'differentiation': "Tecnología que funciona sin conexión a internet y sincroniza datos cuando hay conectividad, ideal para zonas aisladas."
        },
        {
            'summary': "Sistema de rehabilitación física mediante realidad virtual gamificada para pacientes con movilidad reducida.",
            'problem_addressed': "La terapia física tradicional tiene baja adherencia por ser repetitiva y dolorosa, retrasando la recuperación.",
            'solution_proposed': "Juegos de realidad virtual que convierten los ejercicios terapéuticos en experiencias inmersivas y motivadoras.",
            'differentiation': "Algoritmos que adaptan automáticamente la dificultad basándose en el progreso y biomecánica del paciente."
        }
    ],
    'Educación': [
        {
            'summary': "Laboratorios virtuales de ciencias para instituciones educativas con recursos limitados.",
            'problem_addressed': "Millones de estudiantes no tienen acceso a laboratorios científicos para experimentos prácticos esenciales.",
            'solution_proposed': "Simulaciones interactivas en 3D que replican experimentos científicos complejos accesibles desde cualquier dispositivo.",
            'differentiation': "Experiencias multiusuario que permiten colaboración en tiempo real entre estudiantes y profesores remotos."
        },
        {
            'summary': "Plataforma de tutoría entre pares con sistema de microcréditos educativos.",
            'problem_addressed': "El acceso a tutoría personalizada es costoso y está fuera del alcance de muchos estudiantes.",
            'solution_proposed': "Red que conecta estudiantes avanzados con aquellos que necesitan ayuda, usando un sistema de intercambio de conocimientos.",
            'differentiation': "Modelo de economía colaborativa educativa donde los estudiantes ganan créditos enseñando y los usan para recibir ayuda."
        }
    ]
}

def validate_model_with_new_examples(model, encoder, scaler, examples):
    """
    Valida el modelo con nuevos ejemplos manualmente creados.
    
    Esta función preprocesa nuevos ejemplos de la misma manera que los datos de entrenamiento,
    y evalúa el rendimiento del modelo en estos ejemplos.
    
    Args:
        model: El modelo entrenado de RandomForest
        encoder: El OneHotEncoder ajustado a los datos de entrenamiento
        scaler: El StandardScaler ajustado a los datos de entrenamiento
        examples: Diccionario con ejemplos nuevos por categoría
    
    Returns:
        DataFrame con las predicciones y detalles de los ejemplos
    """
    # Crear listas para almacenar los datos de validación
    rows = []
    
    # Procesar cada categoría y sus ejemplos
    for category, category_examples in examples.items():
        for example in category_examples:
            # Generar valores sintéticos aleatorios para las variables no textuales
            # Esto simula diferentes escenarios para cada proyecto
            stage = np.random.choice(['Idea', 'Prototipo', 'MVP', 'Escalando'])  # Etapa del proyecto
            asset_type = np.random.choice(['Equity', 'Deuda', 'Otro'])  # Tipo de activo tokenizado
            asset_existing = np.random.choice([0, 1])  # Si el activo ya existe (binario)
            legal_status = np.random.choice(['Regulado', 'No regulado'])  # Estado legal
            impact_type = np.random.choice(['Social', 'Ambiental', 'Económico'])  # Tipo de impacto
            
            # Datos financieros aleatorios
            estimated_total_cost = np.random.uniform(50000, 1000000)  # Costo estimado
            funding_requested = np.random.uniform(10000, 500000)  # Financiación solicitada
            expected_annual_revenue = np.random.uniform(20000, 2000000)  # Ingresos esperados anuales
            
            # Otros datos operativos
            clients = np.random.choice([0, 1])  # Si tiene clientes o pilotos actualmente
            scalability = np.random.randint(1, 6)  # Puntuación de escalabilidad (1-5)
            
            # Crear un registro completo para este ejemplo
            row = {
                'category': category,  # Categoría del proyecto
                'stage': stage,        # Etapa de desarrollo
                'summary': example['summary'],  # Texto de resumen del proyecto
                'problem_addressed': example['problem_addressed'],  # Problema que aborda
                'solution_proposed': example['solution_proposed'],  # Solución propuesta
                'differentiation': example['differentiation'],  # Diferenciación competitiva
                'tokenized_asset_type': asset_type,  # Tipo de activo tokenizado
                'asset_existing': asset_existing,  # Existencia del activo
                'legal_status': legal_status,  # Estado legal
                'estimated_total_cost_eur': estimated_total_cost,  # Costo total estimado
                'funding_requested_eur': funding_requested,  # Financiación solicitada
                'expected_annual_revenue_eur': expected_annual_revenue,  # Ingresos anuales esperados
                'current_clients_or_pilots': clients,  # Clientes actuales
                'scalability_score': scalability,  # Puntuación de escalabilidad
                'impact_type': impact_type  # Tipo de impacto
            }
            rows.append(row)  # Añadir este registro a la lista
    
    # Crear DataFrame con todos los ejemplos de validación
    validation_data = pd.DataFrame(rows)
    
    # Aplicar el mismo preprocesamiento que se usó para los datos de entrenamiento
    # Es crucial usar los mismos transformadores (encoder, scaler) que se ajustaron con los datos de entrenamiento
    categorical_data = encoder.transform(validation_data[categorical_cols])  # Codificación one-hot
    numeric_data = scaler.transform(validation_data[numeric_cols])  # Normalización
    
    # Procesar textos con BERT igual que en entrenamiento
    text_data = []
    for col in text_cols:
        embeddings = get_bert_embeddings(validation_data[col].tolist())
        text_data.append(embeddings)
    text_data_combined = np.hstack(text_data)  # Combinar todos los embeddings de texto
    
    # Combinar todas las características en formato que el modelo pueda procesar
    X_validation = np.hstack([
        categorical_data,  # Variables categóricas codificadas
        text_data_combined,  # Embeddings de texto
        numeric_data,  # Variables numéricas normalizadas
        validation_data[binary_cols].values  # Variables binarias
    ])
    
    # Hacer predicciones con el modelo entrenado
    predictions = model.predict(X_validation)
    
    # Añadir las predicciones al DataFrame de resultados
    validation_data['predicted_viability'] = predictions
    
    # Calcular la puntuación "real" de viabilidad usando la misma fórmula que en el entrenamiento
    # Esto nos da un punto de referencia para comparar con las predicciones
    stage_score = validation_data['stage'].map({'Idea': 10, 'Prototipo': 20, 'MVP': 30, 'Escalando': 40})
    client_score = validation_data['current_clients_or_pilots'] * 15
    scalability_score = validation_data['scalability_score'] * 5
    revenue_ratio = np.clip(validation_data['expected_annual_revenue_eur'] / (validation_data['estimated_total_cost_eur'] + 1), 0, 5) * 7
    validation_data['actual_viability'] = np.clip(stage_score + client_score + scalability_score + revenue_ratio, 0, 100)
    
    return validation_data  # Devolver resultados de validación

# Validar el modelo con los nuevos ejemplos
validation_results = validate_model_with_new_examples(model, encoder, scaler, validation_examples)

# Mostrar resultados de la validación
print("\nResultados de validación con ejemplos nuevos:")
print(validation_results[['category', 'summary', 'actual_viability', 'predicted_viability']].head())

# Calcular métricas para los ejemplos de validación
validation_mse = mean_squared_error(validation_results['actual_viability'], validation_results['predicted_viability'])
validation_r2 = r2_score(validation_results['actual_viability'], validation_results['predicted_viability'])
print(f"\nMSE en ejemplos de validación: {validation_mse:.2f}")
print(f"R² en ejemplos de validación: {validation_r2:.2f}")
print(validation_results.columns)



MSE: 68.20
R²: 0.80

Características más importantes:
Característica 3088: 0.2454
Característica 3093: 0.1605
Característica 4: 0.1462
Característica 5: 0.1148
Característica 3090: 0.1035
Característica 3091: 0.0838
Característica 6: 0.0182
Característica 3089: 0.0152
Característica 7: 0.0107
Característica 3092: 0.0037

MSE without BERT: 59.78
R² without BERT: 0.82

Resultados de validación con ejemplos nuevos:
  category                                            summary  \
0  Fintech  Plataforma de gestión financiera personal con ...   
1  Fintech  Seguro paramétrico basado en blockchain para p...   
2  Energía  Sistema de almacenamiento de energía basado en...   
3  Energía  Plataforma de comercio de energía P2P para com...   
4    Salud  Plataforma de diagnóstico remoto para zonas ru...   

   actual_viability  predicted_viability  
0        100.000000            94.200785  
1         55.185984            58.402229  
2         86.927590            68.519934  
3         86.868436  

In [12]:
print(validation_results.columns)
print(validation_results.info())

Index(['category', 'stage', 'summary', 'problem_addressed',
       'solution_proposed', 'differentiation', 'tokenized_asset_type',
       'asset_existing', 'legal_status', 'estimated_total_cost_eur',
       'funding_requested_eur', 'expected_annual_revenue_eur',
       'current_clients_or_pilots', 'scalability_score', 'impact_type',
       'predicted_viability', 'actual_viability'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   category                     8 non-null      object 
 1   stage                        8 non-null      object 
 2   summary                      8 non-null      object 
 3   problem_addressed            8 non-null      object 
 4   solution_proposed            8 non-null      object 
 5   differentiation              8 non-null      object 
 6   tokenized_asset_type  

In [5]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from transformers import BertTokenizer, BertModel
import torch
import pickle
import os
from typing import Dict, List, Tuple, Union, Optional

class TextEmbedder:
    """
    Clase para procesar texto utilizando modelos BERT y generar embeddings.
    """
    def __init__(self, model_name='bert-base-uncased', device=None):
        """
        Inicializa el tokenizer y modelo BERT.
        
        Args:
            model_name: Nombre del modelo BERT preentrenado a utilizar
            device: Dispositivo para ejecutar el modelo (None para autodetección)
        """
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        
        # Autodetectar dispositivo si no se especifica
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        
        self.model.to(self.device)
        self.model.eval()  # Establecer modo evaluación
    
    def get_embeddings(self, texts: List[str], max_length: int = 128) -> np.ndarray:
        """
        Convierte una lista de textos en embeddings utilizando BERT.
        
        Args:
            texts: Lista de strings a procesar
            max_length: Longitud máxima de tokens para cada texto
            
        Returns:
            Array NumPy con los embeddings de los textos
        """
        inputs = self.tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            return_tensors='pt', 
            max_length=max_length
        )
        
        # Mover inputs al dispositivo adecuado
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # Extraer el vector [CLS] como representación del texto
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        return embeddings


class ProjectViabilityModel:
    """
    Modelo para predecir la viabilidad de proyectos utilizando características 
    textuales y numéricas.
    """
    def __init__(self):
        """Inicializa el modelo y sus componentes de preprocesamiento."""
        self.text_embedder = TextEmbedder()
        self.encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        self.scaler = StandardScaler()
        self.model = None
        
        # Definir columnas para diferentes tipos de características
        self.categorical_cols = ['category', 'stage', 'tokenized_asset_type', 
                                'legal_status', 'impact_type']
        self.text_cols = ['summary', 'problem_addressed', 
                         'solution_proposed', 'differentiation']
        self.numeric_cols = ['estimated_total_cost_eur', 'funding_requested_eur', 
                            'expected_annual_revenue_eur', 'scalability_score']
        self.binary_cols = ['asset_existing', 'current_clients_or_pilots']
        
    def preprocess_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Preprocesa los datos y extrae características para el modelo.
        
        Args:
            data: DataFrame con los datos a procesar
            
        Returns:
            Tupla con (X, y) donde X son las características y y es la variable objetivo
        """
        # Verificar que todas las columnas necesarias estén presentes
        required_cols = (self.categorical_cols + self.text_cols + 
                         self.numeric_cols + self.binary_cols)
        missing_cols = [col for col in required_cols if col not in data.columns]
        if missing_cols:
            raise ValueError(f"Faltan columnas requeridas en los datos: {missing_cols}")
        
        # Procesar variables categóricas
        categorical_features = self.encoder.transform(data[self.categorical_cols])
        
        # Procesar variables textuales
        text_features = []
        for col in self.text_cols:
            col_embeddings = self.text_embedder.get_embeddings(data[col].tolist())
            text_features.append(col_embeddings)
        text_features_combined = np.hstack(text_features)
        
        # Procesar variables numéricas
        numeric_features = self.scaler.transform(data[self.numeric_cols])
        
        # Combinar todas las características
        X = np.hstack([
            categorical_features,
            text_features_combined,
            numeric_features,
            data[self.binary_cols].values
        ])
        
        # Variable objetivo (puede ser None si no está disponible)
        y = data['viability_score'] if 'viability_score' in data.columns else None
        
        return X, y
    
    def fit(self, data: pd.DataFrame, **model_params) -> Dict:
        """
        Entrena el modelo con los datos proporcionados.
        
        Args:
            data: DataFrame con los datos de entrenamiento
            **model_params: Parámetros para el RandomForestRegressor
            
        Returns:
            Diccionario con métricas de rendimiento del entrenamiento
        """
        # Valores por defecto para los parámetros del modelo
        default_params = {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'random_state': 42
        }
        
        # Actualizar con parámetros proporcionados
        for key, value in model_params.items():
            default_params[key] = value
        
        # Ajustar transformadores con todo el conjunto de datos
        self.encoder.fit(data[self.categorical_cols])
        self.scaler.fit(data[self.numeric_cols])
        
        # Preprocesar datos
        X, y = self.preprocess_data(data)
        
        # Dividir en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Crear y entrenar el modelo
        self.model = RandomForestRegressor(**default_params)
        self.model.fit(X_train, y_train)
        
        # Evaluar el modelo
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Obtener importancia de características
        feature_importances = self.model.feature_importances_
        top_indices = np.argsort(feature_importances)[-10:]
        top_importances = {f"feature_{i}": feature_importances[i] for i in reversed(top_indices)}
        
        return {
            'mse': mse,
            'r2': r2,
            'feature_importances': top_importances
        }
    
    def predict(self, data: pd.DataFrame) -> np.ndarray:
        """
        Realiza predicciones de viabilidad para nuevos proyectos.
        
        Args:
            data: DataFrame con los datos de los proyectos a evaluar
            
        Returns:
            Array con las predicciones de viabilidad
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado. Llame al método fit() primero.")
        
        # Preprocesar datos (ignorando la variable y si existe)
        X, _ = self.preprocess_data(data)
        
        # Realizar predicciones
        predictions = self.model.predict(X)
        
        return predictions
    
    def validate(self, validation_data: pd.DataFrame) -> Dict:
        """
        Valida el modelo con un conjunto de datos de validación.
        
        Args:
            validation_data: DataFrame con datos de validación
            
        Returns:
            Diccionario con métricas y resultados de la validación
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado. Llame al método fit() primero.")
        
        # Realizar predicciones
        predictions = self.predict(validation_data)
        
        # Añadir predicciones al DataFrame
        results_df = validation_data.copy()
        results_df['predicted_viability'] = predictions
        
        # Si existe la columna de viabilidad real, calcular métricas
        metrics = {}
        if 'viability_score' in validation_data.columns:
            mse = mean_squared_error(validation_data['viability_score'], predictions)
            r2 = r2_score(validation_data['viability_score'], predictions)
            metrics = {'mse': mse, 'r2': r2}
        
        return {
            'results': results_df,
            'metrics': metrics,
            'predictions': predictions
        }
    
    def calculate_viability_score(self, data: pd.DataFrame) -> np.ndarray:
        """
        Calcula una puntuación de viabilidad según la fórmula predefinida.
        
        Args:
            data: DataFrame con los datos de los proyectos
            
        Returns:
            Array con las puntuaciones de viabilidad calculadas
        """
        # Factor 1: La etapa más avanzada aumenta la viabilidad
        stage_score = data['stage'].map({'Idea': 10, 'Prototipo': 20, 'MVP': 30, 'Escalando': 40})
        
        # Factor 2: La existencia de clientes aumenta la viabilidad
        client_score = data['current_clients_or_pilots'] * 15
        
        # Factor 3: Mayor escalabilidad aumenta la viabilidad
        scalability_score = data['scalability_score'] * 5
        
        # Factor 4: Mejor ratio ingresos/costos aumenta la viabilidad
        revenue_ratio = np.clip(data['expected_annual_revenue_eur'] / (data['estimated_total_cost_eur'] + 1), 0, 5) * 7
        
        # Calcular puntuación final
        viability_score = stage_score + client_score + scalability_score + revenue_ratio
        
        # Limitar los valores entre 0 y 100
        viability_score = np.clip(viability_score, 0, 100)
        
        return viability_score
    
    def save_model(self, filepath: str):
        """Guarda el modelo y sus componentes para uso futuro."""
        if self.model is None:
            raise ValueError("No hay modelo entrenado para guardar.")
        
        model_data = {
            'model': self.model,
            'encoder': self.encoder,
            'scaler': self.scaler,
            'categorical_cols': self.categorical_cols,
            'text_cols': self.text_cols,
            'numeric_cols': self.numeric_cols,
            'binary_cols': self.binary_cols
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
    
    @classmethod
    def load_model(cls, filepath: str):
        """
        Carga un modelo previamente guardado.
        
        Args:
            filepath: Ruta al archivo del modelo guardado
            
        Returns:
            Instancia de ProjectViabilityModel con el modelo cargado
        """
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        # Crear nueva instancia
        instance = cls()
        
        # Cargar componentes
        instance.model = model_data['model']
        instance.encoder = model_data['encoder']
        instance.scaler = model_data['scaler']
        instance.categorical_cols = model_data['categorical_cols']
        instance.text_cols = model_data['text_cols']
        instance.numeric_cols = model_data['numeric_cols']
        instance.binary_cols = model_data['binary_cols']
        
        return instance


class DataGenerator:
    """
    Clase para generar datos sintéticos de proyectos basados en ejemplos reales.
    """
    def __init__(self, project_examples=None):
        """
        Inicializa el generador de datos con ejemplos de proyectos.
        
        Args:
            project_examples: Diccionario con ejemplos de proyectos por categoría
        """
        # Usar los ejemplos proporcionados o los predeterminados
        self.project_details = project_examples or self._get_default_examples()
        
        # Definir posibles valores para cada característica
        self.categories = list(self.project_details.keys())
        self.stages = ['Idea', 'Prototipo', 'MVP', 'Escalando']
        self.asset_types = ['Equity', 'Deuda', 'Otro']
        self.legal_statuses = ['Regulado', 'No regulado']
        self.impact_types = ['Social', 'Ambiental', 'Económico']
    
    def _get_default_examples(self):
        """
        Proporciona ejemplos predeterminados de proyectos si no se especifican.
        
        Returns:
            Diccionario con ejemplos de proyectos por categoría
        """
        return {
            'Tech': [
                {
                    'summary': "Plataforma de pagos transfronterizos que facilita la transferencia de dinero de manera rápida y segura.",
                    'problem_addressed': "Los pagos internacionales son costosos y lentos, afectando a las pequeñas empresas.",
                    'solution_proposed': "Solución que ofrece tarifas de pago transfronterizas reducidas y procesamiento rápido, utilizando blockchain.",
                    'differentiation': "Utiliza una red de blockchain descentralizada para garantizar la transparencia y baja de costos en pagos."
                },
                # ... otros ejemplos
            ],
            'Energía': [
                {
                    'summary': "Desarrollo de un sistema de energía solar de bajo costo para hogares en zonas rurales.",
                    'problem_addressed': "El acceso a energía limpia en zonas rurales es limitado y costoso.",
                    'solution_proposed': "Sistema de paneles solares con batería de almacenamiento que reduce el costo de energía en áreas rurales.",
                    'differentiation': "Baterías solares desarrolladas con tecnología avanzada que optimizan la eficiencia en lugares remotos."
                },
                # ... otros ejemplos
            ]
            # ... otras categorías
        }
    
    def generate_data(self, n: int = 500) -> pd.DataFrame:
        """
        Genera un conjunto de datos sintéticos basados en ejemplos reales.
        
        Args:
            n: Número de muestras a generar
            
        Returns:
            DataFrame con datos sintéticos
        """
        # Listas para almacenar los datos generados
        category_list = []
        summary_list = []
        problem_list = []
        solution_list = []
        differentiation_list = []
        
        # Generar datos basados en ejemplos reales
        for _ in range(n):
            # Seleccionar una categoría aleatoria
            category = np.random.choice(self.categories)
            category_list.append(category)
            
            # Seleccionar un ejemplo aleatorio de esa categoría
            example = random.choice(self.project_details[category])
            
            # Extraer textos del ejemplo
            summary_list.append(example['summary'])
            problem_list.append(example['problem_addressed'])
            solution_list.append(example['solution_proposed'])
            
            # Verificar y extraer diferenciación si existe
            if 'differentiation' in example:
                differentiation_list.append(example['differentiation'])
            else:
                differentiation_list.append("Ventaja competitiva en el mercado")
        
        # Crear DataFrame con todos los datos
        data = pd.DataFrame({
            'category': category_list,
            'stage': np.random.choice(self.stages, n),
            'summary': summary_list,
            'problem_addressed': problem_list,
            'solution_proposed': solution_list,
            'differentiation': differentiation_list,
            'tokenized_asset_type': np.random.choice(self.asset_types, n),
            'asset_existing': np.random.choice([0, 1], n),
            'legal_status': np.random.choice(self.legal_statuses, n),
            'estimated_total_cost_eur': np.random.uniform(50000, 1000000, n),
            'funding_requested_eur': np.random.uniform(10000, 500000, n),
            'expected_annual_revenue_eur': np.random.uniform(20000, 2000000, n),
            'current_clients_or_pilots': np.random.choice([0, 1], n),
            'scalability_score': np.random.randint(1, 6, n),
            'impact_type': np.random.choice(self.impact_types, n),
        })
        
        # Añadir puntuación de viabilidad
        model = ProjectViabilityModel()
        data['viability_score'] = model.calculate_viability_score(data)
        
        # Añadir algo de ruido a la viabilidad
        data['viability_score'] = data['viability_score'] + np.random.normal(0, 5, n)
        data['viability_score'] = np.clip(data['viability_score'], 0, 100)
        
        return data




# Ejemplo de uso del sistema modular
def main():
    # 1. Generar datos de entrenamiento
    print("Generando datos sintéticos...")
    data_gen = DataGenerator()
    training_data = data_gen.generate_data(n=500)
    print(f"Datos generados: {training_data.shape[0]} ejemplos")
    
    # 2. Crear y entrenar modelo
    print("\nEntrenando modelo...")
    model = ProjectViabilityModel()
    training_metrics = model.fit(training_data)
    
    # 3. Mostrar métricas de entrenamiento
    print(f"MSE en entrenamiento: {training_metrics['mse']:.2f}")
    print(f"R² en entrenamiento: {training_metrics['r2']:.2f}")
    
    print("\nCaracterísticas más importantes:")
    for feature, importance in training_metrics['feature_importances'].items():
        print(f"{feature}: {importance:.4f}")
    
    # 4. Generar datos para validación
    print("\nGenerando datos de validación...")
    validation_data = data_gen.generate_data(n=100)  # Menos ejemplos para validación
    
    # 5. Validar modelo con nuevos datos
    print("Validando modelo...")
    validation_results = model.validate(validation_data)
    
    # 6. Mostrar métricas de validación
    print(f"MSE en validación: {validation_results['metrics']['mse']:.2f}")
    print(f"R² en validación: {validation_results['metrics']['r2']:.2f}")
    
    # 7. Guardar modelo para uso futuro
    model_path = "project_viability_model.pkl"
    model.save_model(model_path)
    print(f"\nModelo guardado en: {model_path}")
    
    # 8. Ejemplo de carga y uso del modelo guardado
    print("\nCargando modelo guardado...")
    loaded_model = ProjectViabilityModel.load_model(model_path)
    
    # 9. Crear un ejemplo específico para probar
    example_project = {
        'category': ['Tech'],
        'stage': ['MVP'],
        'summary': ["Nueva plataforma de inteligencia artificial para optimización de procesos industriales"],
        'problem_addressed': ["Los procesos industriales actuales son ineficientes y generan altos costos operativos"],
        'solution_proposed': ["Software de IA que analiza procesos en tiempo real y recomienda optimizaciones"],
        'differentiation': ["Algoritmos propietarios que reducen costos operativos en un 30% más que competidores"],
        'tokenized_asset_type': ['Equity'],
        'asset_existing': [1],
        'legal_status': ['Regulado'],
        'estimated_total_cost_eur': [350000],
        'funding_requested_eur': [200000],
        'expected_annual_revenue_eur': [1200000],
        'current_clients_or_pilots': [1],
        'scalability_score': [5],
        'impact_type': ['Económico']
    }
    example_df = pd.DataFrame(example_project)
    
    # 10. Predecir viabilidad del ejemplo
    prediction = loaded_model.predict(example_df)[0]
    print(f"Predicción de viabilidad para el proyecto ejemplo: {prediction:.2f}/100")

if __name__ == "__main__":
    main()

Generando datos sintéticos...
Datos generados: 500 ejemplos

Entrenando modelo...
MSE en entrenamiento: 47.43
R² en entrenamiento: 0.85

Características más importantes:
feature_3088: 0.2102
feature_3086: 0.1911
feature_3: 0.1351
feature_3089: 0.1225
feature_3091: 0.1197
feature_2: 0.1047
feature_5: 0.0359
feature_4: 0.0262
feature_3087: 0.0231
feature_7: 0.0042

Generando datos de validación...
Validando modelo...
MSE en validación: 66.58
R² en validación: 0.79

Modelo guardado en: project_viability_model.pkl

Cargando modelo guardado...
Predicción de viabilidad para el proyecto ejemplo: 83.95/100


In [3]:
# from model_modules import ProjectViabilityModel, DataGenerator, TextEmbedder

# Ejemplo de uso del sistema modular
def main():
    # 1. Generar datos de entrenamiento
    print("Generando datos sintéticos...")
    data_gen = DataGenerator()
    training_data = data_gen.generate_data(n=500)
    print(f"Datos generados: {training_data.shape[0]} ejemplos")
    
    # 2. Crear y entrenar modelo
    print("\nEntrenando modelo...")
    model = ProjectViabilityModel()
    training_metrics = model.fit(training_data)
    
    # 3. Mostrar métricas de entrenamiento
    print(f"MSE en entrenamiento: {training_metrics['mse']:.2f}")
    print(f"R² en entrenamiento: {training_metrics['r2']:.2f}")
    
    print("\nCaracterísticas más importantes:")
    for feature, importance in training_metrics['feature_importances'].items():
        print(f"{feature}: {importance:.4f}")
    
    # 4. Generar datos para validación
    print("\nGenerando datos de validación...")
    validation_data = data_gen.generate_data(n=100)  # Menos ejemplos para validación
    
    # 5. Validar modelo con nuevos datos
    print("Validando modelo...")
    validation_results = model.validate(validation_data)
    
    # 6. Mostrar métricas de validación
    print(f"MSE en validación: {validation_results['metrics']['mse']:.2f}")
    print(f"R² en validación: {validation_results['metrics']['r2']:.2f}")
    
    # 7. Guardar modelo para uso futuro
    model_path = "project_viability_model.pkl"
    model.save_model(model_path)
    print(f"\nModelo guardado en: {model_path}")
    
    # 8. Ejemplo de carga y uso del modelo guardado
    print("\nCargando modelo guardado...")
    loaded_model = ProjectViabilityModel.load_model(model_path)
    
    # 9. Crear un ejemplo específico para probar
    example_project = {
        'category': ['Tech'],
        'stage': ['MVP'],
        'summary': ["Nueva plataforma de inteligencia artificial para optimización de procesos industriales"],
        'problem_addressed': ["Los procesos industriales actuales son ineficientes y generan altos costos operativos"],
        'solution_proposed': ["Software de IA que analiza procesos en tiempo real y recomienda optimizaciones"],
        'differentiation': ["Algoritmos propietarios que reducen costos operativos en un 30% más que competidores"],
        'tokenized_asset_type': ['Equity'],
        'asset_existing': [1],
        'legal_status': ['Regulado'],
        'estimated_total_cost_eur': [350000],
        'funding_requested_eur': [200000],
        'expected_annual_revenue_eur': [1200000],
        'current_clients_or_pilots': [1],
        'scalability_score': [5],
        'impact_type': ['Económico']
    }
    example_df = pd.DataFrame(example_project)
    
    # 10. Predecir viabilidad del ejemplo
    prediction = loaded_model.predict(example_df)[0]
    print(f"Predicción de viabilidad para el proyecto ejemplo: {prediction:.2f}/100")

if __name__ == "__main__":
    main()

Generando datos sintéticos...


NameError: name 'DataGenerator' is not defined